In [ ]:
from rayleigh_diagnostics import AZ_Avgs, build_file_list, plot_azav, streamfunction
import matplotlib.pyplot as plt
import pylab
import numpy
import glob
%matplotlib inline

dir_list = ['./ek1em1/','./ek1em2/','./ek1em3/','./ek1em4/','./ek1em5/']

ek_name = [r'$E = 10^{-1}$',r'$E = 10^{-2}$',r'$E = 10^{-3}$',r'$E = 10^{-4}$',r'$E = 10^{-5}$']

In [ ]:
figdpi=300
sizetuple=(5.5*3,3*3)

tsize = 20     # title font size
cbfsize = 10   # colorbar font size
dir_size = len(dir_list)
fig, ax = plt.subplots(ncols=dir_size,figsize=sizetuple,dpi=figdpi)
plt.rcParams.update({'font.size': 14})

for k in dir_list:
    w = sorted(glob.glob(k + 'AZ_Avgs/*'))
    files = [w[-1]]

    az = AZ_Avgs(files[0],path='')
    nfiles = len(files)
    tcount=0
    for i in range(nfiles):
        az=AZ_Avgs(files[i],path='')

        if (i == 0):
            nr = az.nr
            ntheta = az.ntheta
            nq = az.nq
            azavg=numpy.zeros((ntheta,nr,nq),dtype='float64')

        for j in range(az.niter):
            azavg[:,:,:] += az.vals[:,:,:,j]
            tcount+=1
    azavg = azavg*(1.0/tcount)

    lut = az.lut
    vr = azavg[:,:,lut[1]]
    vtheta = azavg[:,:,lut[2]]
    vphi = azavg[:,:,lut[3]]
    rhovr = azavg[:,:,lut[201]]
    rhovtheta = azavg[:,:,lut[202]]
    temperature = azavg[:,:,lut[501]]
    radius = az.radius
    costheta = az.costheta
    sintheta = az.sintheta

    #Subtrace the ell=0 component from temperature at each radius
    for i in range(nr):
        temperature[:,i]=temperature[:,i] - numpy.mean(temperature[:,i])

    #Convert v_phi to an Angular velocity
    omega=numpy.zeros((ntheta,nr))
    for i in range(nr):
        omega[:,i]=vphi[:,i]/(radius[i]*sintheta[:])

    psi = streamfunction(rhovr,rhovtheta,radius,costheta,order=0)
    #contours of mass flux are overplotted on the streamfunction PSI
    rhovm = numpy.sqrt(rhovr**2+rhovtheta**2)*numpy.sign(psi) 
    

    #temperature
    #ax1 = f1.add_subplot(1,3,1)
    units = '(nondimensional)'
    for h in range(5):
        if k is dir_list[h]:
            numbering = h
        else:
            pass
    plot_azav(fig,ax[numbering],psi,radius,costheta,sintheta,mycmap='RdYlBu_r',boundsfactor = 2, 
          boundstype='rms', units=units, fontsize = cbfsize)
    ax[numbering].set_title(ek_name[numbering],fontsize=tsize)

plt.tight_layout()
savefile = './plots/AZ_Avgs_massflux.pdf'
#print('Saving figure to: ', savefile)
plt.savefig(savefile, bbox_inches='tight')
plt.show()


In [ ]:
import glob

for k in dir_list:
    f = sorted(glob.glob(k+'AZ_Avgs/*'))

print(f[-1])